<a href="https://colab.research.google.com/github/cloner174/mr.DGA/blob/main/mr_DGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
data_path = "data/Faze1/input/dataset.csv"

In [ ]:
%pip install imutils opencv-python dlib opencv-python-headless pillow matplotlib face_recognition

In [9]:
import pandas as pd
import cv2
import pandas as pd
import face_recognition
import numpy as np
import os

In [10]:
def strings_with_number_inside( data,
                                    column_range_starts : int = 0,
                                    column_range_ends : int = None,
                                    seperator = None):
        """
        Converts cells with type String contains a series of numbers separated with spaces to a Dictionary and Gives it back.
        """
        start = column_range_starts
        end = int( data.shape[1] ) if column_range_ends is None else column_range_ends  #Columns
        # Split string and convert to uint8 numpy array
        new_data = {}
        if isinstance( data , pd.DataFrame ) :
            for i in range( data.shape[0] ) :
                temp_new_data = []
                for j in range( start, end ) :
                    # for each cell
                    # Any exception should be modify by column_range_starts and column_range_ends!
                    cells_real_values = np.array(data.iloc[i, j].split(sep = seperator), dtype='uint8')
                    temp_new_data.append(cells_real_values)
                new_data[i] = temp_new_data

            return new_data
        else:
            if isinstance( data , np.array ) :
                pass
            else:
                raise TypeError( " data just could be  PandasDataFrame  or  NumpyArray  ! ")

In [11]:
# Load data
csv_path = 'data/Faze1/input/dataset.csv'
data = pd.read_csv(csv_path)

In [22]:
data.to_csv('data/output/data-pix2.csv', index= False, compression = {'method': 'zip', 'compresslevel': 8 })

In [12]:
# Path to save images
face_directory = 'data/output/face_directory/'
non_face_directory = 'data/output/non_face/'

In [15]:
data_pix = strings_with_number_inside(data, column_range_starts = 1)

In [17]:
data_pix_df = pd.DataFrame(data_pix)

In [16]:
data_pix.to_csv('data/output/data-pix.csv', index= False, comprees = s)

AttributeError: 'dict' object has no attribute 'to_csv'

In [20]:
48*48

2304

In [14]:


# Define the image size
image_size = (48, 48)  # height, width

# Ensure directories exist
os.makedirs(face_directory, exist_ok=True)
os.makedirs(non_face_directory, exist_ok=True)

# loop to process images
for index, row in data_pix.items():
    # Convert the row to a grayscale image
    image_array = np.array(row, dtype=np.uint8).reshape(image_size[0], image_size[1])

    # In face_recognition, the image should be in rgb even if it's grayscale
    image_rgb = np.stack((image_array,) * 3, axis=-1)

    # Detect faces in the image using face_recognition
    face_locations = face_recognition.face_locations(image_rgb, model='cnn')  # using CNN model

    # Process each face found
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image_array[top:bottom, left:right]  # Cropping the grayscale image directly
        cv2.imwrite(os.path.join(face_directory, f'face_{index}.jpg'), face_image)

    # If no faces are detected, save the full image to the non-face directory
    if len(face_locations) == 0:
        cv2.imwrite(os.path.join(non_face_directory, f'non_face_{index}.jpg'), image_array)

print("Face cropping and saving completed.")


KeyboardInterrupt: 

In [10]:
in_face_folder = os.listdir('/content/data/face_directory')

In [12]:
non_face_folder = os.listdir('/content/data/non_face')

In [11]:
len(in_face_folder)

31519

In [13]:
len(non_face_folder)

4368

In [ ]:
!zip -r '/content/face_directory.zip' '/content/data/face_directory/'

In [ ]:
!zip -r '/content/non_face.zip' '/content/data/non_face'